# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import gmaps.datasets
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# import csv from part 1 and load to a dataframe
cities_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv", encoding="utf-8")
cities_data_df

# Check data types
print(cities_data_df.dtypes)

Unnamed: 0      int64
City           object
Latitude      float64
Longitude     float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object


In [3]:
# Drop anything that may be blank
cities_data_df = cities_data_df.dropna()
cities_data_df = cities_data_df.drop(columns=['Unnamed: 0'])
cities_data_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,illoqqortoormiut,31.45,12.98,69.31,40,0,1.66,LY,1595730153
1,mizdah,-8.93,125.41,70.09,54,78,5.44,TL,1595730153
2,airai,-23.12,-134.97,70.48,80,18,20.94,PF,1595730153
3,rikitea,-46.19,168.86,52.63,68,42,6.04,NZ,1595730153
4,mataura,-53.15,-70.92,24.80,85,49,10.29,CL,1595730002


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [5]:
# Store latitude and longitude into locations
locations = cities_data_df[["Latitude","Longitude"]]

# Store humidity
# humidity = cities_data_df[["Humidity"]].astype(float)
humidity = pd.to_numeric(cities_data_df['Humidity'], downcast='float')



In [6]:
# Create heatmap layer
coordinates=(0.0,0.0)

figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=coordinates, zoom_level=1, map_type='SATELLITE')

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, point_radius=8)
fig.add_layer(heatmap_layer)
plt.savefig("output_data/IdealHeatMap.png")

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='600px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Create dataframe to narrow down to ideal weather conditions
ideal_weather = cities_data_df.loc[((cities_data_df['Max Temp'] > 72) & (cities_data_df['Max Temp'] < 80)) & \
                                  (cities_data_df['Wind Speed'] < 10) & (cities_data_df['Cloudiness'] == 0)]
ideal_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,klaksvik,64.75,177.48,73.40,69,0,6.71,RU,1595730155
33,minsk,42.60,-73.97,78.01,86,0,4.23,US,1595730158
71,nishihara,36.21,57.68,78.04,32,0,3.27,IR,1595730163
89,vila franca do campo,39.37,49.25,77.47,66,0,4.52,AZ,1595730166
100,dikson,36.77,-4.10,77.00,43,0,0.54,ES,1595730168
154,anju,-21.15,149.20,79.00,57,0,6.93,AU,1595729889
258,port elizabeth,-20.02,148.23,79.00,69,0,8.05,AU,1595730192
343,plettenberg bay,37.30,-77.29,79.00,86,0,3.33,US,1595730204
369,tutoia,35.47,-6.03,75.20,64,0,2.24,MA,1595730208
385,curvelo,62.08,150.48,74.95,26,0,8.99,RU,1595730189


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# create dataframe for hotel info
hotel_df = pd.DataFrame(ideal_weather)

hotel_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,klaksvik,64.75,177.48,73.40,69,0,6.71,RU,1595730155
33,minsk,42.60,-73.97,78.01,86,0,4.23,US,1595730158
71,nishihara,36.21,57.68,78.04,32,0,3.27,IR,1595730163
89,vila franca do campo,39.37,49.25,77.47,66,0,4.52,AZ,1595730166
100,dikson,36.77,-4.10,77.00,43,0,0.54,ES,1595730168


In [11]:
# create params dict that will be updated with new coordinates for each iteration
target_type = "lodging"
params = {
    "key": g_key,
    "radius": 5000,
    "type": target_type
}

In [13]:
# iterate through rows to find closest hotel
for index, row in hotel_df.iterrows():
    #get latitude and longitude from datafra,e
    lat = row["Latitude"]
    long = row["Longitude"]
    
    params["location"] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    hotel_info = requests.get(base_url, params = params)
    hotel_info = hotel_info.json()
   
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_info["results"][0]["name"]
      
    except(KeyError, IndexError):
        print(f"No hotel found. Skipping...")
        
hotel_df

No hotel found. Skipping...
No hotel found. Skipping...


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,klaksvik,64.75,177.48,73.40,69,0,6.71,RU,1595730155,"Гостиница ""Анадырь"""
33,minsk,42.60,-73.97,78.01,86,0,4.23,US,1595730158,NaN
71,nishihara,36.21,57.68,78.04,32,0,3.27,IR,1595730163,پارکینگ وسایل سنگین جنوب
89,vila franca do campo,39.37,49.25,77.47,66,0,4.52,AZ,1595730166,Göl hotel
100,dikson,36.77,-4.10,77.00,43,0,0.54,ES,1595730168,Hotel Torremar
154,anju,-21.15,149.20,79.00,57,0,6.93,AU,1595729889,International Lodge Motel
258,port elizabeth,-20.02,148.23,79.00,69,0,8.05,AU,1595730192,Castle Motor Lodge Motel
343,plettenberg bay,37.30,-77.29,79.00,86,0,3.33,US,1595730204,Best Western Plus Hopewell Inn
369,tutoia,35.47,-6.03,75.20,64,0,2.24,MA,1595730208,Hotel Zelis
385,curvelo,62.08,150.48,74.95,26,0,8.99,RU,1595730189,NaN


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

plt.savefig("output_data/IdealHeatMapWithMarkers.png")

# Display figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='600px'))

<Figure size 432x288 with 0 Axes>